Import Python Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Get Suburb details in a DataFrame

In [5]:
df_sub = pd.read_excel("suburb_post_region.xlsx")
df_sub.head()

,suburb,Post Codes,region
0,Sydney City,2000,Central & Inner Metropolitan
1,Ultimo,2007,Central & Inner Metropolitan
2,Chippendale,2008,Central & Inner Metropolitan
3,Pyrmont,2009,Central & Inner Metropolitan
4,Surry Hills,2010,Central & Inner Metropolitan


Get Crime Data suburb wise

In [2]:
df_sub_crime  = pd.read_excel("suburb_crime.xlsx")
df_sub_crime.head()

,suburb,crime
0,Sydney City,M
1,Ultimo,M
2,Chippendale,L
3,Pyrmont,H
4,Surry Hills,H


Get Income Details

In [3]:
df_sub_income  = pd.read_excel("suburb_income.xlsx")
df_sub_income.head()

,suburb,income
0,Sydney City,H
1,Ultimo,H
2,Chippendale,M
3,Pyrmont,M
4,Surry Hills,M


Merge all dataframes

In [7]:
df_subcrime = pd.merge(df_sub, df_sub_crime, on='suburb', how='inner')
df_subcrime_income = pd.merge(df_subcrime, df_sub_income, on='suburb', how='inner')
print(df_subcrime_income.head())
df_subcrime_income.shape


        suburb  Post Codes                        region crime income
0  Sydney City        2000  Central & Inner Metropolitan     M      H
1       Ultimo        2007  Central & Inner Metropolitan     M      H
2  Chippendale        2008  Central & Inner Metropolitan     L      M
3      Pyrmont        2009  Central & Inner Metropolitan     H      M
4  Surry Hills        2010  Central & Inner Metropolitan     H      M


(202, 5)

Function to get category 

In [8]:
 def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Define Foursquare credentials

In [9]:
CLIENT_ID = 'SRB50TFUBSODQKLA4EZJOAMJOLPV2O0BAD22VQJJ0TDQJMQM' # your Foursquare ID
CLIENT_SECRET = 'Y3UKS3WJKCXJ2EZ4LDLTVCTHNHESGDCLG4XIWNHKADPZL35R' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
search_query = 'Indian Restaurant'
radius = 500

function to get restaurant details of all suburbs

In [10]:
def getNearbyRest(suburbs, crimes, incomes, radius=500):
    all_rest  =pd.DataFrame()  
    for name, crime, income in zip(suburbs, crimes, incomes):
        print(name)
        # find latititude, longitude
        try:
            address = name +  ', NSW, Australia'
            geolocator = Nominatim()
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        
            
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
            
            
            # make the GET request
            results = requests.get(url).json()
            sub_rest = results['response']['venues']
            # tranform venues into a dataframe
            df_sub_rest  = json_normalize(sub_rest)
            # filter the category for each row
            df_sub_rest['categories'] = df_sub_rest.apply(get_category_type, axis=1)

            # clean column names by keeping only last term
            df_sub_rest.columns = [column.split('.')[-1] for column in df_sub_rest.columns]
            df_sub_rest['suburb'] =  name
            df_sub_rest['latitude'] = latitude
            df_sub_rest['longitude'] = longitude
            df_sub_rest['crime']     = crime
            df_sub_rest['income']    = income
            df_sub_rest_filt = df_sub_rest[['suburb', 'crime', 'income', 'name', 'latitude', 'longitude', 'categories', 'lat', 'lng']]
            all_rest  = pd.concat( [all_rest, df_sub_rest_filt])
            #print(df_sub_rest.head())
            #print(all_rest.head())
        except:
             continue         
    return(all_rest)

Get All Restaurant Name and Ctegories

In [11]:
sydney_rest = getNearbyRest(suburbs = df_subcrime_income['suburb'],
                             crimes = df_subcrime_income['crime'],    
                             incomes = df_subcrime_income['income'])
sydney_rest.shape

Sydney City


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Ultimo
Chippendale
Pyrmont
Surry Hills
Kings Cross
Alexandria
Redfern
Waterloo
Rosebery
Botany
Mascot
Paddington
Bondi Junction
Bellevue Hill
Waverley
Woollahra
Bondi
Edgecliff
Double Bay
Rose Bay
Vaucluse
Randwick
Kingsford
Kensington
Coogee
Pagewood
Matraville
Glebe
Annandale
Rozelle
Leichhardt
Balmain
Newtown
Erskineville
St Peters
Haberfield
Five Dock
Drummoyne
Stanmore
Petersham
Camperdown
North Sydney
Milsons Point
Cammeray
Northbridge
Artarmon
Crows Nest
Lane Cove
Chatswood
Willoughby
Roseville
Lindfield
Killara
Gordon
Pymble
Turramurra
St Ives
Wahroona
Hornsby
Mount Colah
Mount Kuring-gai
Berowra Waters
Terrey Hills
Belrose
Frenchs Forest
Forestville
Mosman
Neutral Bay
Cremorne
Seaforth
Balgowlah
Manly
Harbord
Collaroy
Dee Why
Brookvale
Narrabeen
Warriewood
Mona Vale
Bayview
Newport
Avalon
Hunters Hill
Gladesville
Ryde
North Ryde
Meadowbank
Ermington
Rydalmere
Telopea
Carlingford
Beecroft
Pennant Hills
Epping
Eastwood
W. Pennant Hills
Cherrybrook
Homebush Bay
Silverwater
Sydney

(795, 9)

In [12]:
sydney_rest.head()


,suburb,crime,income,name,latitude,longitude,categories,lat,lng
0,Sydney City,M,H,Bennelong Restaurant,-33.854816,151.216454,Australian Restaurant,-33.857564,151.214794
1,Sydney City,M,H,Aqua Luna Bar & Restaurant,-33.854816,151.216454,Cocktail Bar,-33.856578,151.211186
2,Sydney City,M,H,Aria Restaurant,-33.854816,151.216454,Australian Restaurant,-33.858933,151.213316
0,Ultimo,M,H,Superbowl Chinese Restaurant,-33.879473,151.198435,Chinese Restaurant,-33.878317,151.204067
1,Ultimo,M,H,The Apprentice Restaurant,-33.879473,151.198435,Australian Restaurant,-33.881575,151.200867


Add Indian/ Asian columns based on Name / Categories of Restaurants

In [13]:
#cond  = "suburb_table2.Neigbourhood == 'Not assigned' "
sydney_rest['indian'] = np.where(  (sydney_rest.categories.str.find('Indian') >= 0) |  
                                    (sydney_rest.name.str.find('Indian') >= 0 ) |      
                                    (sydney_rest.name.str.find('India') >= 0)  , 1,0)
sydney_rest['asian'] = np.where(  (sydney_rest.categories.str.find('Chinese') >= 0) |  
                                   (sydney_rest.name.str.find('Chinese') >= 0 ) |  
                                  (sydney_rest.categories.str.find('Thai') >= 0) |  
                                   (sydney_rest.name.str.find('Thai') >= 0 ) | 
                                   (sydney_rest.categories.str.find('Japanese') >= 0) |  
                                   (sydney_rest.name.str.find('Japan') >= 0 ) |
                                   (sydney_rest.categories.str.find('Korean') >= 0) |  
                                   (sydney_rest.name.str.find('Korean') >= 0 ) |
                                   (sydney_rest.categories.str.find('Malaysian') >= 0) |  
                                   (sydney_rest.name.str.find('Malaysian') >= 0 ) ,1,0)
  
sydney_rest.head()

,suburb,crime,income,name,latitude,longitude,categories,lat,lng,indian,asian
0,Sydney City,M,H,Bennelong Restaurant,-33.854816,151.216454,Australian Restaurant,-33.857564,151.214794,0,0
1,Sydney City,M,H,Aqua Luna Bar & Restaurant,-33.854816,151.216454,Cocktail Bar,-33.856578,151.211186,0,0
2,Sydney City,M,H,Aria Restaurant,-33.854816,151.216454,Australian Restaurant,-33.858933,151.213316,0,0
0,Ultimo,M,H,Superbowl Chinese Restaurant,-33.879473,151.198435,Chinese Restaurant,-33.878317,151.204067,0,1
1,Ultimo,M,H,The Apprentice Restaurant,-33.879473,151.198435,Australian Restaurant,-33.881575,151.200867,0,0


In [14]:
# add all indian and asian restaurnts numbers
sydney_rest_a = sydney_rest.groupby(['suburb']).sum()
sydney_rest_a.head()

,latitude,longitude,lat,lng,indian,asian
suburb,,,,,,
Annandale,-67.762448,302.341995,-67.766821,302.336109,1,0
Artarmon,-135.236038,604.741096,-135.233784,604.736944,1,2
Ashfield,-610.010606,2720.293425,-609.999033,2720.247552,2,11
Auburn,-846.232550,3775.820265,-846.259948,3775.819185,1,10
Avalon,-100.909512,453.987090,-100.911261,453.988151,0,3


Get required columns for modelling

In [15]:
sydney_rest_model  = sydney_rest_a[['indian', 'asian']]
sydney_rest_model.head()

,indian,asian
suburb,,
Annandale,1,0
Artarmon,1,2
Ashfield,2,11
Auburn,1,10
Avalon,0,3


Form clustetrs using k-Means algorithm

In [16]:
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sydney_rest_model)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([0, 2, 1, 1, 2], dtype=int32)

In [17]:
# add clustering labels
sydney_rest_model['Cluster Labels'] = kmeans.labels_
sydney_rest_model.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,indian,asian,Cluster Labels
suburb,,,
Annandale,1,0,0
Artarmon,1,2,2
Ashfield,2,11,1
Auburn,1,10,1
Avalon,0,3,2


In [18]:
# Add Latitude and Longitude
sydney_m = pd.merge(sydney_rest_model, sydney_rest[['suburb','longitude', 'latitude']], on='suburb', how='inner')
sydney_m.drop_duplicates(inplace=True)
sydney_m.head()                             

,suburb,indian,asian,Cluster Labels,longitude,latitude
0,Annandale,1,0,0,151.170998,-33.881224
2,Artarmon,1,2,2,151.185274,-33.809010
6,Ashfield,2,11,1,151.127412,-33.889478
24,Auburn,1,10,1,151.032811,-33.849302
49,Avalon,0,3,2,151.329030,-33.636504


In [19]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the 
import folium # map rendering library


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    pandas-0.23.4              |   py36hf8a1672_0        27.8 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        28.4 MB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001       conda-forge
    branca:  0.3.1-py_0            conda-forge
    folium:  0.5.0-py_0            conda-forge
    v

In [20]:
# get latitude and longitude for Sydney
address = 'Sydney, Australia'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


-33.8548157 151.2164539


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sydney_m['latitude'], sydney_m['longitude'], sydney_m['suburb'], sydney_m['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
sydney_m.loc[sydney_m['Cluster Labels'] == 0 ]

,suburb,indian,asian,Cluster Labels,longitude,latitude
6,Ashfield,2,11,0,151.127412,-33.889478
24,Auburn,1,10,0,151.032811,-33.849302
99,Beverly Hills,0,10,0,151.081734,-33.949744
123,Bondi Junction,3,7,0,151.250123,-33.893003
153,Campsie,1,9,0,151.103213,-33.914389
212,Chippendale,2,8,0,151.199821,-33.886329
257,Crows Nest,4,11,0,151.203609,-33.825565
403,Kings Cross,6,9,0,151.222423,-33.874601
433,Kingsford,0,11,0,151.227231,-33.921592
473,Lidcombe,1,7,0,151.045677,-33.863065


In [38]:
sydney_m.loc[sydney_m['Cluster Labels'] == 1]

,suburb,indian,asian,Cluster Labels,longitude,latitude
0,Annandale,1,0,1,151.170998,-33.881224
58,Balmain,2,1,1,151.180735,-33.858399
92,Beecroft,1,1,1,151.065747,-33.749763
94,Belfield,1,0,1,151.086001,-33.904516
95,Belmore,0,2,1,151.087894,-33.919407
98,Belrose,0,1,1,151.219144,-33.730009
118,Blakehurst,0,0,1,151.108560,-33.989852
119,Bondi,0,2,1,151.263333,-33.893056
135,Botany,0,0,1,151.199064,-33.947829
137,Brookvale,0,2,1,151.270702,-33.762885


In [39]:
sydney_m.loc[sydney_m['Cluster Labels'] == 2]

,suburb,indian,asian,Cluster Labels,longitude,latitude
302,Eastwood,0,25,2,151.084444,-33.790000
736,Ultimo,0,24,2,151.198435,-33.879473


In [40]:
sydney_m.loc[sydney_m['Cluster Labels'] == 3]

,suburb,indian,asian,Cluster Labels,longitude,latitude
2,Artarmon,1,2,3,151.185274,-33.809010
49,Avalon,0,3,3,151.329030,-33.636504
52,Balgowlah,1,3,3,151.260826,-33.794442
69,Bankstown,1,3,3,151.033588,-33.917190
88,Baulkham Hills,1,2,3,150.991404,-33.761120
139,Burwood,0,4,3,151.102481,-33.885809
170,Castle Hill,2,3,3,151.007172,-33.731532
208,Chester Hill,0,3,3,151.001111,-33.881667
242,Cremorne,0,5,3,151.225433,-33.826008
250,Cronulla,1,3,3,151.152385,-34.057526


In [41]:
sydney_m.loc[sydney_m['Cluster Labels'] == 4]

,suburb,indian,asian,Cluster Labels,longitude,latitude
178,Chatswood,5,16,4,151.181857,-33.79448
